In [1]:
# | default_exp classes.DomoDataflow_Execution

In [2]:
# | exporti

from __future__ import annotations

from enum import Enum
from dataclasses import dataclass, field
import httpx

import datetime as dt
import domolibrary.utils.convert as ct
import domolibrary.utils.DictDot as util_dd

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.dataflow as dataflow_routes
from nbdev.showdoc import patch_to, show_doc

In [3]:
@dataclass
class DomoDataflow_Execution:
    auth: dmda.DomoAuth = field(repr=False)
    id: str
    dataflow_id: str
    dataflow_execution_id: str
    dataflow_version: str

    begin_time: dt.datetime
    end_time: dt.datetime
    last_updated: dt.datetime

    is_failed: bool
    state: str
    activation_type: str
    data_processor: str
    telemetry: dict
    execution_stats: dict

    @classmethod
    def _from_json(cls, de_obj, auth: dmda.DomoAuth):
        return cls(
            auth=auth,
            id=de_obj["id"],
            dataflow_id=de_obj["onboardFlowId"],
            dataflow_execution_id=de_obj["dapDataFlowExecutionId"],
            dataflow_version=de_obj.get("dataFlowVersion"),
            begin_time=ct.convert_epoch_millisecond_to_datetime(de_obj["beginTime"]),
            end_time=ct.convert_epoch_millisecond_to_datetime(de_obj["endTime"]),
            last_updated=ct.convert_epoch_millisecond_to_datetime(
                de_obj["lastUpdated"]
            ),
            is_failed=de_obj["failed"],
            state=de_obj["state"],
            activation_type=de_obj["activationType"],
            data_processor=de_obj["dataProcessor"],
            telemetry=de_obj.get("telemetry"),
            execution_stats={
                "total_bytes_written": de_obj.get("totalBytesWritten", 0),
                "total_rows_read": de_obj.get("totalRowsRead", 0),
                "total_bytes_read": de_obj.get("totalBytesRead", 0),
                "mean_download_rate_kbps": de_obj.get("meanDownloadRateKbps", 0),
                "total_rows_written": de_obj.get("totalRowsWritten", 0),
            },
        )
        return cls(auth=auth, **de_obj)


In [4]:
@patch_to(DomoDataflow_Execution, cls_method = True)
async def get_by_id(cls, 
    auth: dmda.DomoAuth,
    dataflow_id: int,
    execution_id : int,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
    return_raw: bool = False):

    res = await dataflow_routes.get_dataflow_execution_by_id(
        auth = auth,
        dataflow_id = dataflow_id,
        execution_id = execution_id,
        debug_api = debug_api,
        debug_num_stacks_to_drop = debug_num_stacks_to_drop,
        parent_class = parent_class,
        session = session
    )

    if return_raw:
        return res
        
    return cls._from_json(auth = auth, de_obj = res.response)
    

In [5]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await DomoDataflow_Execution.get_by_id(
    dataflow_id=70, execution_id=433238, auth=token_auth,
    return_raw = True
)

from domolibrary.classes.DomoDataflow import DomoDataflow_Action, DomoDataflow_ActionExecuted

action_ls = res.response['actionResults']
domo_actions = [DomoDataflow_ActionExecuted._from_json(action_obj) for action_obj in action_ls]

domo_actions[0].__dict__

TypeError: DomoDataflow_ActionExecuted.__post_init__() takes 0 positional arguments but 1 was given

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()